# Capstone Project Week 3 (Part 3)

# Segmenting and Clustering Neighborhoods in Toronto

#### by We You Toh

## Introduction

This is a 3-part assignment, which I aim to guide the reader through the process of using data to discover some interesting things about the neighborhoods in Toronto. In the first part, I will prepare a dataframe that consist of the neighborhoods in Toronto and their respective latitude and longitude values. I will also use the Folium library to visualize the neighborhoods in Toronto.

In the second part, I will be using the Foursquare API to explore and analyze the neighborhoods in Toronto. I will use the **explore** function to get the most common venue categories in each neighborhood.

In the last part, I will use the neighborhood and venue data retrieved from the second part to group the neighborhoods into clusters. I will use the *k*-means clustering algorithm to gain some insights about the neighborhoods in Toronto.  I will once again use the Folium library to visualize the emerging clusters among the neighborhoods in Toronto.

>*__Note:__ This notebook is publicly shared on github repository. The Folium interactive maps and some Markdown features don't display on github the same way as they do on a local host. To interact with these features, it may be necessary to download the jupyter notebook and host it locally. If you wish, you may re-run the whole notebook on your own. You may also wish to provide your own Foursquare API credentials.*
>
> *__Tip:__ If you run the notebook on a local host, set the notebook to 'Trusted' to enable javascript display, otherwise the Folium maps may not display properly.*

<div class="alert alert-block alert-info" style="margin-top: 10px"></div>

## Table of Contents

### Part 1
1. [**Scraping Postal Codes of Toronto and Prepare Dataset**](#item1) 

### Part 2
2. [**Explore Neighborhoods in Toronto**](#item2)  
3. [**Analyze Each Neighborhood**](#item3)  

### Part 3
4. [**Cluster Neighborhoods**](#item4)  
5. [**Examine Clusters**](#item5)  

<div class="alert alert-block alert-info" style="margin-top: 0px"></div>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
# For web-scraping
from bs4 import BeautifulSoup # library for efficient web data extraction
import requests # library to handle requests

# For data-handling
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# For geographical-related tasks
!conda install -c conda-forge geopy --yes # uncomment this line if geopy have not been installed
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if folium has not been installed.
import folium # map rendering library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')


Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.17.0                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


<a id='item1'></a>

## 1. Scraping Postal Codes of Toronto and Preparing Dataset

In [2]:
# Retrieving postal codes from the internet
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
soup = BeautifulSoup(page.content, "lxml")

In [3]:
# Creating the dataframe
data = []
table = soup.find('table', attrs={'class':"wikitable sortable"})
rows = table.find_all('tr')

for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols]) if cols else None # Get rid of empty rows

df = pd.DataFrame(data, columns = ['PostalCode','Borough','Neighborhood'])

df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Data Wrangling

In [4]:
# Removing postal codes not assigned to a borough
df = df[df.Borough != 'Not assigned']

# Replace unassigned neighborhoods with its borough's name.
df[df.Neighborhood == 'Not assigned'] = df.assign(Neighborhood = df.Borough)


In [5]:
# A quick survey over the data frame
print("Number of postal codes: ", len(df.PostalCode.unique()))
print("Number of boroughs: ", len(df.Borough.unique()))
print("Number of neighborhoods: ", len(df.Neighborhood.unique()))

Number of postal codes:  103
Number of boroughs:  11
Number of neighborhoods:  210


In [6]:
# Group neighborhoods by their postal codes and boroughs
df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()

df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
df.shape

(103, 3)

Neighborhood has a total of 11 boroughs and 210 neighborhoods in 103 postal codes. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 11 boroughs and the neighborhoods that exist in each borough as well as the the latitude and longitude coordinates of each postal code.

The data set for the latitude and longitude coordinates for each postal code shall be retrieved from the site provided in the Coursera assignment: https://cocl.us/Geospatial_data.

In [8]:
!wget -q -O 'toronto_data.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


#### Load and explore the data

Next, let's load the data.

In [9]:
postalcode_latlong = pd.read_csv('toronto_data.csv')

Let's take a quick look at the data.

In [10]:
postalcode_latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Tranform the data into a *pandas* dataframe

In [11]:
# Merging the latitude and longitude coordinates into a new dataframe.
neighborhoods = df.join(postalcode_latlong.set_index('Postal Code'), on='PostalCode')

neighborhoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


And make sure that the dataset has all 11 boroughs and 103 postal codes.

In [12]:
print('The dataframe has {} boroughs and {} postal codes.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]))

The dataframe has 11 boroughs and 103 postal codes.


#### Use geopy library to get the latitude and longitude values of Toronto.

In [13]:
address = 'Toronto, Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Ontario are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Toronto, Ontario are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [14]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], 
                                           neighborhoods['Longitude'], 
                                           neighborhoods['Borough'], 
                                           neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

<a id='item2'></a>

## 2. Explore Neighborhoods in Toronto

#### Define Foursquare Credentials and Version

In [15]:
# The code was removed by Watson Studio for sharing.

Foursquare API credentials placed in this code segment: CLIENT_ID, CLIENT_SECRET, VERSION


#### Let's explore the first neighborhood by its postal code prefix in our dataframe.

Get the neighborhood's name.

In [16]:
neighborhoods.loc[0, 'Neighborhood']

'Rouge, Malvern'

Get the neighborhood's latitude and longitude values.

In [17]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'PostalCode'] # neighborhood's postal code prefix

print('Latitude and longitude values of postal code prefix {} are {}, {}.'.format(
    neighborhood_name,
    neighborhood_latitude,
    neighborhood_longitude))

Latitude and longitude values of postal code prefix M1B are 43.806686299999996, -79.19435340000001.


#### Now, let's get the top 100 venues that are in postal code prefix M1B (Rouge and Malvern) area within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [18]:
# Set up the GET request URL
radius = 500
LIMIT = 100

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    neighborhood_latitude,
    neighborhood_longitude,
    VERSION,
    radius,
    LIMIT)

Send the GET request and examine the resutls

In [19]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5bbc15734c1f6719cd3dc6ea'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4bb6b9446edc76b0d771311c-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/fastfood_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16e941735',
         'name': 'Fast Food Restaurant',
         'pluralName': 'Fast Food Restaurants',
         'primary': True,
         'shortName': 'Fast Food'}],
       'id': '4bb6b9446edc76b0d771311c',
       'location': {'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Morningside & Sheppard',
        'distance': 387,
        'formattedAddress': ['Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.80744841934756,
          'ln

We will define the **get_category_type** function to extract the information from the results.

In [20]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [21]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056
1,Interprovincial Group,Print Shop,43.805630,-79.200378


In this exploratory exercise, there are only two venues returned for the neighborhood(s) having the M1B postal code prefix.

In [22]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [24]:
# Storing venues in Toronto

toronto_venues = getNearbyVenues(neighborhoods.Neighborhood,neighborhoods.Latitude,neighborhoods.Longitude)

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, Steeles West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The D

#### Let's check the size of the resulting dataframe

In [25]:
print(toronto_venues.shape)
toronto_venues.head()

(2255, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge, Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place


Let's check how many venues were returned for each neighborhood

In [26]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",11,11,11,11,11,11
"Alderwood, Long Branch",10,10,10,10,10,10
"Bathurst Manor, Downsview North, Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",26,26,26,26,26,26
Berczy Park,55,55,55,55,55,55


There are many neighborhoods that have less than 10 venues. Let's remove those neighborhoods from the dataframe.

In [27]:
toronto_venues = toronto_venues.groupby('Neighborhood').filter(lambda x: len(x)>=10)

#### Let's find out how many unique categories can be curated from all the returned venues

In [28]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 249 uniques categories.


#### Let's find out how many neighborhoods have 10 or more venues.

In [29]:
print(
    'There are {} neighborhoods remaining after filtering out neighborhoods with less than 10 venues.'
    .format(len(toronto_venues.Neighborhood.unique())))

There are 48 neighborhoods remaining after filtering out neighborhoods with less than 10 venues.


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [30]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Restaurant,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
72,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"L'Amoreaux West, Steeles West",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

And let's examine the new dataframe size.

In [31]:
toronto_onehot.shape

(2042, 249)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [32]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Restaurant,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,0.00,0.010000,0.010000,0.000000,0.020000,0.000000,0.0000,0.000000,0.0000,0.000000,0.020000,0.000000,0.030000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.01,0.020000,0.000000,0.000000,0.000000,0.00,0.02

#### Let's confirm the new size

In [33]:
toronto_grouped.shape

(48, 249)

#### Let's print each neighborhood along with the top 5 most common venues

In [34]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.06
2              Gym  0.04
3  Thai Restaurant  0.04
4       Steakhouse  0.04


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                 venue  freq
0        Grocery Store  0.18
1  Japanese Restaurant  0.09
2             Pharmacy  0.09
3       Discount Store  0.09
4          Pizza Place  0.09


----Alderwood, Long Branch----
                venue  freq
0         Pizza Place   0.2
1                 Gym   0.1
2  Athletics & Sports   0.1
3         Coffee Shop   0.1
4                 Pub   0.1


----Bathurst Manor, Downsview North, Wilson Heights----
              venue  freq
0       Coffee Shop  0.11
1     Shopping Mall  0.06
2       Supermarket  0.06
3        Restaurant  0.06
4  Sushi Restaurant  0.06


----Bedford Park, Lawrence Manor East----
                  venue  freq
0           Coffee Shop  0.08
1  F

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [36]:
num_top_venues = 10

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,Gym,American Restaurant,Restaurant,Hotel,Bar,Gastropub
1,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Fried Chicken Joint,Pharmacy,Pizza Place,Sandwich Place,Discount Store,Fast Food Restaurant,Japanese Restaurant,Beer Store,Coffee Shop
2,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Pharmacy,Sandwich Place,Bank,Athletics & Sports,Pub,Skating Rink,Gym,Concert Hall
3,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Fried Chicken Joint,Pharmacy,Pizza Place,Deli / Bodega,Bridal Shop,Restaurant,Diner,Sandwich Place,Shopping Mall
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Fast Food Restaurant,Grocery Store,Pizza Place,Sandwich Place,Juice Bar,Liquor Store,Japanese Restaurant,Restaurant
5,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Beer Bar,Steakhouse,Restaurant,Cheese Shop,Café,Farmers Market
6,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Breakfast Spot,Café,Performing Arts Venue,Burrito Place,Caribbean Restaurant,Climbing Gym,Pet Store,Office,Stadium
7,Business reply mail Processing Centre969 Eastern,Skate Park,Restaurant,Auto Workshop,Burrito Place,Brewery,Butcher,Fast Food Restaurant,Farmers Market,Spa,Pizza Place
8,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Plane,Boutique,Harbor / Marina,Airport,Airport Food Court,Airport Gate
9,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Pizza Place,Convenience Store,Pub,Indian Restaurant,Italian Restaurant,Chinese Restaurant,Bakery,Café


<a id='item4'></a>

## 4. Cluster Neighborhoods

I have chosen to run *k*-means to cluster the neighborhood into 7 clusters after some experimentation. I have chosen 7 clusters as it seems to bring certain neighborhoods out distinctly.

In [37]:
# set number of clusters
kclusters = 7

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 5, 6, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 5, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       6, 3], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [38]:
toronto_merged = neighborhoods

# add clustering labels
neighborhoods_venues_sorted['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), 
                                     on='Neighborhood', 
                                     how='inner')

toronto_merged # check the merged dataset

,PostalCode,Borough,Neighborhood,Latitude,Longitude,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
15,M1W,Scarborough,"L'Amoreaux West, Steeles West",43.799525,-79.318389,Fast Food Restaurant,Chinese Restaurant,Grocery Store,Breakfast Spot,Coffee Shop,Sandwich Place,Pharmacy,Pizza Place,Japanese Restaurant,Cosmetics Shop,5
18,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,Clothing Store,Fast Food Restaurant,Coffee Shop,Women's Store,Toy / Game Store,Cosmetics Shop,Bus Station,Bakery,Food Court,Tea Room,0
22,M2N,North York,Willowdale South,43.770120,-79.408493,Ramen Restaurant,Restaurant,Pizza Place,Café,Sandwich Place,Grocery Store,Korean Restaurant,Hotel,Ice Cream Shop,Indonesian Restaurant,0
27,M3C,North York,"Flemingdon Park, Don Mills South",43.725900,-79.340923,Asian Restaurant,Gym,Coffee Shop,Beer Store,Grocery Store,General Entertainment,Italian Restaurant,Japanese Restaurant,Fast Food Restaurant,Dim Sum Restaurant,0
28,M3H,North York,"Bathurst Manor, Downsview North, Wilson Heights",43.754328,-79.442259,Coffee Shop,Fried Chicken Joint,Pharmacy,Pizza Place,Deli / Bodega,Bridal Shop,Restaurant,Diner,Sandwich Place,Shopping Mall,0
35,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937,Pizza Place,Fast Food Restaurant,Pharmacy,Athletics & Sports,Rock Climbing Spot,Intersection,Bank,Café,Gastropub,Pet Store,6
36,M4C,East York,Woodbine Heights,43.695344,-79.318389,Park,Skating Rink,Asian Restaurant,Cosmetics Shop,Curling Ice,Intersection,Athletics & Sports,Beer Store,Video Store,Gastropub,3
38,M4G,East York,Leaside,43.709060,-79.363452,Coffee Shop,Sporting Goods Shop,Grocery Store,Burger Joint,Breakfast Spot,Fish & Chips Shop,Bank,Smoothie Shop,Beer Store,Shopping Mall,0
39,M4H,East York,Thorncliffe Park,43.705369,-79.349372,Indian Restaurant,Grocery Store,Sandwich Place,Bank,Burger Joint,Bus Station,Coffee Shop,Discount Store,Gym,Gym / Fitness Center,0
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Grocery Store,Fruit & Vegetable Store,Pizza Place,Liquor Store,Juice Bar,0


In [39]:
toronto_merged.shape

(48, 16)

Finally, let's visualize the resulting clusters

In [40]:
# create map
map_clusters = folium.Map(location=[toronto_merged.Latitude.mean(), toronto_merged.Longitude.mean()], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], 
                                  toronto_merged['Longitude'], 
                                  toronto_merged['Neighborhood'],
                                  toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, let's examine each cluster and determine the discriminating venue categories that distinguish each cluster.

#### Cluster 1

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, 
                   toronto_merged.columns[[0,1,2] + list(range(5, toronto_merged.shape[1]-1))]]

,PostalCode,Borough,Neighborhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,M2J,North York,"Fairview, Henry Farm, Oriole",Clothing Store,Fast Food Restaurant,Coffee Shop,Women's Store,Toy / Game Store,Cosmetics Shop,Bus Station,Bakery,Food Court,Tea Room
22,M2N,North York,Willowdale South,Ramen Restaurant,Restaurant,Pizza Place,Café,Sandwich Place,Grocery Store,Korean Restaurant,Hotel,Ice Cream Shop,Indonesian Restaurant
27,M3C,North York,"Flemingdon Park, Don Mills South",Asian Restaurant,Gym,Coffee Shop,Beer Store,Grocery Store,General Entertainment,Italian Restaurant,Japanese Restaurant,Fast Food Restaurant,Dim Sum Restaurant
28,M3H,North York,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Fried Chicken Joint,Pharmacy,Pizza Place,Deli / Bodega,Bridal Shop,Restaurant,Diner,Sandwich Place,Shopping Mall
38,M4G,East York,Leaside,Coffee Shop,Sporting Goods Shop,Grocery Store,Burger Joint,Breakfast Spot,Fish & Chips Shop,Bank,Smoothie Shop,Beer Store,Shopping Mall
39,M4H,East York,Thorncliffe Park,Indian Restaurant,Grocery Store,Sandwich Place,Bank,Burger Joint,Bus Station,Coffee Shop,Discount Store,Gym,Gym / Fitness Center
41,M4K,East Toronto,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Grocery Store,Fruit & Vegetable Store,Pizza Place,Liquor Store,Juice Bar
42,M4L,East Toronto,"The Beaches West, India Bazaar",Sandwich Place,Park,Food & Drink Shop,Board Shop,Liquor Store,Brewery,Italian Restaurant,Burger Joint,Fast Food Restaurant,Burrito Place
43,M4M,East Toronto,Studio District,Café,Coffee Shop,Gastropub,Bakery,Italian Restaurant,American Restaurant,Comfort Food Restaurant,Seafood Restaurant,Sandwich Place,Breakfast Spot
46,M4R,Central Toronto,North Toronto West,Clothing Store,Sporting Goods Shop,Coffee Shop,Health & Beauty Service,Fast Food Restaurant,Diner,Dessert Shop,Mexican Restaurant,Miscellaneous Shop,Park


This cluster features a whole variety of venues -- from restaurants to entertainment venues, to stores selling various kinds of goods. From the map, these neighborhoods cluster around the city center and towards the north of the city.

#### Cluster 2

In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, 
                   toronto_merged.columns[[0,1,2] + list(range(5, toronto_merged.shape[1]-1))]]

,PostalCode,Borough,Neighborhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
75,M6G,Downtown Toronto,Christie,Grocery Store,Café,Park,Athletics & Sports,Nightclub,Restaurant,Italian Restaurant,Diner,Convenience Store,Baby Store
76,M6H,West Toronto,"Dovercourt Village, Dufferin",Supermarket,Bakery,Brewery,Café,Bank,Bar,Middle Eastern Restaurant,Portuguese Restaurant,Discount Store,Fast Food Restaurant
82,M6P,West Toronto,"High Park, The Junction South",Bar,Café,Mexican Restaurant,Grocery Store,Fried Chicken Joint,Sandwich Place,Fast Food Restaurant,Italian Restaurant,Diner,Flea Market
88,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto",Café,Sandwich Place,Pizza Place,Mexican Restaurant,Bakery,Fish & Chips Shop,Fried Chicken Joint,Fast Food Restaurant,Restaurant,Pharmacy


It is hard to tell from the assortment of stores in this cluster. All neighborhoods in this cluster feature cafés and restaurants, as well as some form of convenience stores, e.g. grocery store, supermarket or pharmacy, despite the variety of labels.

#### Cluster 3

In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, 
                   toronto_merged.columns[[0,1,2] + list(range(5, toronto_merged.shape[1]-1))]]

,PostalCode,Borough,Neighborhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Plane,Boutique,Harbor / Marina,Airport,Airport Food Court,Airport Gate


This cluster clearly features the airport and the harbor quite a bit.

#### Cluster 4

In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, 
                   toronto_merged.columns[[0,1,2] + list(range(5, toronto_merged.shape[1]-1))]]

,PostalCode,Borough,Neighborhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,M4C,East York,Woodbine Heights,Park,Skating Rink,Asian Restaurant,Cosmetics Shop,Curling Ice,Intersection,Athletics & Sports,Beer Store,Video Store,Gastropub


This cluster highlights the neighborhood of Woodbine Heights in East York that seems to feature public spaces/sports venue, such as the park, skating rink and curling center, more than the other neighborhoods.

#### Cluster 5

In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, 
                   toronto_merged.columns[[0,1,2] + list(range(5, toronto_merged.shape[1]-1))]]

,PostalCode,Borough,Neighborhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
71,M6A,North York,"Lawrence Heights, Lawrence Manor",Clothing Store,Furniture / Home Store,Event Space,Accessories Store,Coffee Shop,Vietnamese Restaurant,Shoe Store,Fraternity House,Boutique,Dive Bar


This cluster seems to highlight a couple of neighborhoods in North York that features event space and fraternity house. The prominence of commercial stores in this cluster stands out also.

#### Cluster 6

In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, 
                   toronto_merged.columns[[0,1,2] + list(range(5, toronto_merged.shape[1]-1))]]

,PostalCode,Borough,Neighborhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,M1W,Scarborough,"L'Amoreaux West, Steeles West",Fast Food Restaurant,Chinese Restaurant,Grocery Store,Breakfast Spot,Coffee Shop,Sandwich Place,Pharmacy,Pizza Place,Japanese Restaurant,Cosmetics Shop
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Fried Chicken Joint,Pharmacy,Pizza Place,Sandwich Place,Discount Store,Fast Food Restaurant,Japanese Restaurant,Beer Store,Coffee Shop


There are only two neighborhoods (by postal code prefix) in this cluster. It is hard to say if there is anything that stands out in this cluster except that it is not as close to the city center as the neighborhoods featured in Cluster 1. Out of the list of ten most common venues in each neighborhood in Cluster 6, seven labels are identical.

#### Cluster 7

In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, 
                   toronto_merged.columns[[0,1,2] + list(range(5, toronto_merged.shape[1]-1))]]

,PostalCode,Borough,Neighborhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,M4B,East York,"Woodbine Gardens, Parkview Hill",Pizza Place,Fast Food Restaurant,Pharmacy,Athletics & Sports,Rock Climbing Spot,Intersection,Bank,Café,Gastropub,Pet Store
89,M8W,Etobicoke,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Pharmacy,Sandwich Place,Bank,Athletics & Sports,Pub,Skating Rink,Gym,Concert Hall


There are only two neighborhoods (by postal code prefix) in this cluster. What stands out in this cluster is the prominence of sports venues. Both neighborhoods also have pizza place, pharmacy, bank, coffee shop/café, pub/gastropub in common.

### Thank you for checking out this notebook!

This concludes the final part of the assignment. I hope you have found this notebook interesting!

This notebook was created by We You Toh.